In [1]:
from stats import run_algo

In [2]:
filename = 'result.txt'

In [3]:
import os
run_algo(alg='CBS', dest_file='result.txt',
         map_path=os.path.join(os.curdir, 'data', 'maps', 'mapf', 'random-32-32-10.map'),
         scen_path=os.path.join(os.curdir, 'data', 'scens', 'mapf', 'random-32-32-10-even-1.scen'),
         tasks_count=10,
         test_num=10)

True

In [5]:
from collections import defaultdict
def parse_path_file(path_file_content):
    agents_path = defaultdict(list)
    agent = None
    lines = path_file_content.split('\n')
    for line in lines:
        if line == '':
            break
        nums = list(map(int, line.split(' ')))
        if len(nums) == 1:
            agent = int(nums[0])
        else:
            x, y, time = nums
            agents_path[agent].append((x, y, time))
    return agents_path

def read_paths(filename=filename):
    with open(filename, 'r') as path_file:
        path_file_content = path_file.read()
        agents_path = parse_path_file(path_file_content)
    return dict(agents_path)

In [9]:
import tqdm
# test num takes index in range(0, num_repetitions)
def success_rate(alg_name, map_name, scen_name, num_agents, num_repetitions):
    rate = 0
    for i in tqdm.tqdm(range(num_repetitions)):
        done = run_algo(alg=alg_name, dest_file=filename,
                 map_path=os.path.join(os.curdir, 'data', 'maps', 'mapf', map_name),
                 scen_path=os.path.join(os.curdir, 'data', 'scens', 'mapf', scen_name),
                 tasks_count=num_agents,
                 test_num=i)
        rate += int(done)
    return rate / num_repetitions


In [11]:
random_map = 'random-32-32-10.map'
random_scen = 'random-32-32-10-even-1.scen'
success_rate('CBS', random_map, random_scen, 10, 100)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [02:13<00:00,  1.34s/it]


0.99

In [12]:
random_map = 'random-32-32-10.map'
random_scen = 'random-32-32-10-even-1.scen'
success_rate('CBS', random_map, random_scen, 1, 100)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 60.31it/s]


1.0

In [13]:
random_map = 'random-32-32-10.map'
random_scen = 'random-32-32-10-even-1.scen'
success_rate('CBS', random_map, random_scen, 20, 100)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [13:55<00:00,  8.36s/it]


0.94

In [ ]:
# build success_rate stats
rate_list = [] 
N = 10
for num_actors in [5, 10, 20, 30, 40, 50]:
    rate = success_rate('CBS', random_map, random_scen, num_actors, N)
    print(f'Num actors :{num_actors}, Rate: {rate:.2f}%')
    rate_list.append(rate)

 40%|████████████████████████████████████████████                                                                  | 4/10 [00:00<00:00, 39.22it/s]

Num actors :5, Rate: 1.00%


 20%|██████████████████████                                                                                        | 2/10 [00:00<00:00, 16.26it/s]

Num actors :10, Rate: 1.00%


 10%|███████████                                                                                                   | 1/10 [00:00<00:01,  5.92it/s]

Num actors :20, Rate: 1.00%
